In [ ]:
!pip install rasterio matplotlib wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rasterio-1.3.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20.9 MB)
  Using cached wandb-0.13.9-py2.py3-none-any.whl (2.0 MB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached GitPython-3.1.30-py3-none-any.whl (184 kB)
  Using cached setproctitle-1.3.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31 kB)
  Using cached sentry_sdk-1.13.0-py2.py3-none-any.whl (177 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)


In [ ]:
def mount_bucket(bucket_name):
  """
  This fn authenticates your google account and mounts to you Google Cloud Platform bucket
  """
  !echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
  !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
  !sudo apt-get -y -q update
  !sudo apt-get -y -q install gcsfuse

  from google.colab import auth
  auth.authenticate_user()
  !mkdir -p gcp_bucket 
  !gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {bucket_name} gcp_bucket

mount_bucket('finlandforest')
%cd drive/MyDrive/finland_forest/

deb http://packages.cloud.google.com/apt gcsfuse-focal main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  50416      0 --:--:-- --:--:-- --:--:-- 50416
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [4,999 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [1,7

In [ ]:
%load_ext autoreload
%autoreload 2

%cd drive/MyDrive/finland_forest

/content/drive/MyDrive/finland_forest


In [ ]:
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import data

import warnings
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)  # our rasters are not georeferenced
warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')

PATH = '/content/drive/MyDrive/finland_forest/'

In [ ]:
features = pd.read_csv(f'{PATH}data/features_proc.csv')
train_features = features[features['split'] == 'train']
test_features = features[features['split'] == 'test']

In [ ]:
rasterio.open('/content/gcp_bucket/train_features/93dd0d32_agbm.tif').read()

array([[[  0.  ,   0.  ,   0.  , ..., 169.72,  96.03,  25.2 ],
        [ 42.62,  80.79,  80.79, ...,  26.95,  47.67,  35.67],
        [ 42.62,  80.79,  80.79, ...,  26.95,  47.67,  35.67],
        ...,
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ]]],
      dtype=float32)

In [ ]:
features

,filename,chip_id,satellite,split,month,corresponding_agbm,image_filepath,label_filepath
0,93dd0d32_S2_07.tif,93dd0d32,S2,train,4,93dd0d32_agbm.tif,gcp_bucket/train_features/93dd0d32_S2_07.tif,gcp_bucket/train_features/93dd0d32_agbm.tif
1,832b55bd_S1_04.tif,832b55bd,S1,train,1,832b55bd_agbm.tif,gcp_bucket/train_features/832b55bd_S1_04.tif,gcp_bucket/train_features/832b55bd_agbm.tif
2,a6d62d7f_S2_00.tif,a6d62d7f,S2,test,9,a6d62d7f_agbm.tif,gcp_bucket/train_features/a6d62d7f_S2_00.tif,gcp_bucket/train_features/a6d62d7f_agbm.tif
3,a74d5ce1_S1_10.tif,a74d5ce1,S1,train,7,a74d5ce1_agbm.tif,gcp_bucket/train_features/a74d5ce1_S1_10.tif,gcp_bucket/train_features/a74d5ce1_agbm.tif
4,404571ba_S1_04.tif,404571ba,S1,train,1,404571ba_agbm.tif,gcp_bucket/train_features/404571ba_S1_04.tif,gcp_bucket/train_features/404571ba_agbm.tif
...,...,...,...,...,...,...,...,...
252421,d3e6581d_S1_11.tif,d3e6581d,S1,test,8,d3e6581d_agbm.tif,gcp_bucket/train_features/d3e6581d_S1_11.tif,gcp_bucket/train_features/d3e6581d_agbm.tif
252422,e7535960_S1_07.tif,e7535960,S1,train,4,e7535960_agbm.tif,gcp_bucket/train_features/e7535960_S1_07.tif,gcp_bucket/train_features/e7535960_agbm.tif
252423,25a25054_S2_08.tif,25a25054,S2,train,5,25a25054_agbm.tif,gcp_bucket/train_features/25a25054_S2_08.tif,gcp_bucket/train_features/25a25054_agbm.tif
252424,508cbf2e_S2_04.tif,508cbf2e,S2,test,1,508cbf2e_agbm.tif,gcp_bucket/train_features/508cbf2e_S2_04.tif,gcp_bucket/train_features/508cbf2e_agbm.tif


In [ ]:
%pwd

'/content/drive/MyDrive/finland_forest'